In [1]:
import torch
import torchinfo
import timm
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from dataset import *

C:\Users\admin\anaconda3\envs\tf-torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [273]:
import torchmetrics

In [355]:
task = "multiclass"
top_k = 1
accuracy = torchmetrics.classification.Accuracy(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
precision = torchmetrics.Precision(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
recall = torchmetrics.Recall(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
f1 = torchmetrics.F1Score(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
stat = torchmetrics.StatScores(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")

In [356]:
preds = torch.tensor([[-0.0129, -0.3175, -0.0683,  0.2080,  0.1129,  0.0553,  0.0140,  0.0251,
         -0.1527,  0.0622],
        [ 0.1304,  0.1785,  0.0883,  0.5066, -0.1161, -0.1496, -0.2035,  0.0709,
          0.2420,  0.3981],
        [-0.0213, -0.0850, -0.1686,  0.1521,  0.2414, -0.0275,  0.0066,  0.1848,
         -0.1766,  0.0321],
        [ 0.3734,  0.3885, -0.2002, -0.3148, -0.2604,  0.3571, -0.7075, -0.2140,
          0.2051,  0.7275],
        [-0.1535, -0.1203, -0.0069,  0.2753,  0.3317, -0.3602, -0.1358,  0.1422,
         -0.0302,  0.3005],
        [ 0.2874,  0.1921,  0.2205,  0.3036, -0.1055, -0.1325,  0.0322,  0.4317,
          0.1032, -0.0067],
        [ 0.0414, -0.1034, -0.1293,  0.0489,  0.3273,  0.1023, -0.1438,  0.1175,
         -0.0843,  0.1121],
        [-0.1684, -0.0975, -0.1321,  0.1957,  0.1615,  0.1282, -0.2379,  0.0721,
         -0.1267,  0.1032],
        [ 0.0425, -0.1601, -0.0020,  0.1313,  0.1604,  0.1026,  0.0792,  0.1182,
         -0.0664,  0.0552],
        [-0.0258, -0.2268, -0.0416,  0.0605,  0.1985,  0.0211, -0.2180,  0.1641,
         -0.0347, -0.0319]]).to("cuda")
target = torch.tensor([3, 7, 0, 1, 3, 8, 9, 8, 3, 6]).to("cuda")
print(stat(preds, target))
print(accuracy(preds, target))
print(precision(preds, target))
print(recall(preds, target))
print(f1(preds, target))

tensor([ 1,  9, 81,  9, 10], device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')


In [351]:
print(stat.compute())
print(accuracy.compute())
print(precision.compute())
print(recall.compute())
print(f1.compute())

tensor([ 4, 46, 44,  6, 10], device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.0800, device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.1333, device='cuda:0')


In [276]:
root_path = f"C:/Users/conon/Documents/Datasets/cifar10/"
train_path = os.path.join(root_path, "train")
val_path = os.path.join(root_path, "test")

train_labels_path = os.path.join(root_path, "train_labels.csv")
val_labels_path = os.path.join(root_path, "test_labels.csv")

In [138]:
train = datasets.CIFAR10(root=root_path, train=True, download=True)
val = datasets.CIFAR10(root=root_path, train=False, download=True)

for i, img in enumerate(train.data):
     np.save(os.path.join(root_path + "train", str(i)), img)
pd.DataFrame({"label": train.targets}).to_csv(train_labels_path, index=True)
for i, img in enumerate(val.data):
     np.save(os.path.join(root_path + "test", str(i)), img)
pd.DataFrame({"label": val.targets}).to_csv(val_labels_path, index=True)

Files already downloaded and verified
Files already downloaded and verified


In [166]:
timm.list_models("vgg*", pretrained=True)

['vgg11.tv_in1k',
 'vgg11_bn.tv_in1k',
 'vgg13.tv_in1k',
 'vgg13_bn.tv_in1k',
 'vgg16.tv_in1k',
 'vgg16_bn.tv_in1k',
 'vgg19.tv_in1k',
 'vgg19_bn.tv_in1k']

In [180]:
model = timm.create_model("vgg11", pretrained=True, num_classes=10, in_chans=3)

In [181]:
torchinfo.summary(model, input_size=[1, 3, 32, 32], depth=5)

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [1, 64, 32, 32]           1,792
│    └─ReLU: 2-2                         [1, 64, 32, 32]           --
│    └─MaxPool2d: 2-3                    [1, 64, 16, 16]           --
│    └─Conv2d: 2-4                       [1, 128, 16, 16]          73,856
│    └─ReLU: 2-5                         [1, 128, 16, 16]          --
│    └─MaxPool2d: 2-6                    [1, 128, 8, 8]            --
│    └─Conv2d: 2-7                       [1, 256, 8, 8]            295,168
│    └─ReLU: 2-8                         [1, 256, 8, 8]            --
│    └─Conv2d: 2-9                       [1, 256, 8, 8]            590,080
│    └─ReLU: 2-10                        [1, 256, 8, 8]            --
│    └─MaxPool2d: 2-11                   [1, 256, 4, 4]            -

In [170]:
param_name = [name for name,_ in model.named_parameters()] # All parameters name
layer_name = [name for name,_ in model.named_modules()] # All layers name

In [173]:
param_name

['features.0.weight',
 'features.0.bias',
 'features.3.weight',
 'features.3.bias',
 'features.6.weight',
 'features.6.bias',
 'features.8.weight',
 'features.8.bias',
 'features.11.weight',
 'features.11.bias',
 'features.13.weight',
 'features.13.bias',
 'features.16.weight',
 'features.16.bias',
 'features.18.weight',
 'features.18.bias',
 'pre_logits.fc1.weight',
 'pre_logits.fc1.bias',
 'pre_logits.fc2.weight',
 'pre_logits.fc2.bias',
 'head.fc.weight',
 'head.fc.bias']

In [177]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [182]:
requires_grad = False
for i, module in enumerate(model.named_parameters()):
    name, param = module
    if "pre_logits" in name:
        requires_grad = True
    param.requires_grad = requires_grad
for name, param in model.named_parameters():
    print(name, param.requires_grad)

features.0.weight False
features.0.bias False
features.3.weight False
features.3.bias False
features.6.weight False
features.6.bias False
features.8.weight False
features.8.bias False
features.11.weight False
features.11.bias False
features.13.weight False
features.13.bias False
features.16.weight False
features.16.bias False
features.18.weight False
features.18.bias False
pre_logits.fc1.weight True
pre_logits.fc1.bias True
pre_logits.fc2.weight True
pre_logits.fc2.bias True
head.fc.weight True
head.fc.bias True


In [183]:
pd.DataFrame.from_dict(model.default_cfg, orient="index")

,0
url,
hf_hub_id,timm/vgg11.tv_in1k
architecture,vgg11
tag,tv_in1k
custom_load,False
input_size,"(3, 224, 224)"
fixed_input_size,False
interpolation,bilinear
crop_pct,0.875
crop_mode,center


In [70]:
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [71]:
transforms

Compose(
    Resize(size=284, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    ToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [74]:
model.default_cfg["input_size"][1:]

(256, 256)

In [82]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

transforms.Resize(size=256, interpolation=InterpolationMode.BICUBIC)


Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)

In [10]:
# Custom final layer
num_in_features = model.get_classifier().in_features
model.head.fc = nn.Sequential(
    nn.BatchNorm1d(num_in_features),
    nn.Linear(in_features=num_in_features, out_features=512, bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=10, bias=False)
)

In [114]:
model.eval()
model(torch.randn(1, 3, 256, 256)).shape

torch.Size([1, 10])

In [115]:
model.feature_info

[{'num_chs': 64, 'reduction': 4, 'module': 'stages.0'},
 {'num_chs': 128, 'reduction': 8, 'module': 'stages.1'},
 {'num_chs': 256, 'reduction': 16, 'module': 'stages.2'},
 {'num_chs': 512, 'reduction': 32, 'module': 'stages.3'}]